### Enfoque 1, Estadísticos de alto orden, medias y desviaciones estándar (Enfoque tradicional)


Por cada muestra crear un vector de (1,80), donde 80 son:
- Las medias de los 20 coeficientes MFCC de todas las ventanas.
- Las desviaciones estándar de los 20 coeficientes MFCC de todas las ventanas.
- Los Kurtosis de los 20 coeficientes MFCC de todas las ventanas.
- Los skewness de los 20 coeficientes MFCC de todas las ventanas


In [70]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing
from scipy.stats import skew

from scipy.stats import kurtosis

In [49]:
#PATHSAMPLES = '../data/training_list_test_temporal.txt'
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES= 1
#SOUNDS =np.array(['yes','no','right','five','nine'])
SOUNDS =np.array(['off','five','left'])

In [50]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    
    
        #Encode etiqueta del audio,speaker y la palabra
    lePath = preprocessing.LabelEncoder()
    leSpeaker = preprocessing.LabelEncoder()
    leWord =  preprocessing.LabelEncoder()
    
    lePath.fit(path_word_speakerId[:,0])
    leSpeaker.fit(path_word_speakerId[:,2])
    leWord.fit(path_word_speakerId[:,1])
    
    return matrixMajor

In [51]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [52]:
path_word_speakerId.shape

(3, 3)

In [123]:
def build_features(path_word_speakerId, n_mfcc,NSAMPLES, PATHSAMPLES, SOUNDS):
    N_rows = NSAMPLES*len(SOUNDS)*n_mfcc
    N_colums = n_mfcc +3# número de columnas del mfcc + 3 columnas (la palabra y el speaker id, path al que pertenece)
    # el path es necesario ya que estamos teniendo un enfoque de multiples instancias.
    path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)
    matrix_features = np.empty((1,n_mfcc*4 +3)) # inicialmente

    #Encode etiqueta del audio,speaker y la palabra
    lePath = preprocessing.LabelEncoder()
    leSpeaker = preprocessing.LabelEncoder()
    leWord =  preprocessing.LabelEncoder()
    
    lePath.fit(path_word_speakerId[:,0])
    leSpeaker.fit(path_word_speakerId[:,2])
    leWord.fit(path_word_speakerId[:,1])
    
        
    for sound_info in path_word_speakerId:
        audio_path = '../'+sound_info[0]
        x , sr = librosa.load(audio_path)
        x_normalize=sk.preprocessing.minmax_scale(x, axis=0)       
        mfccs = librosa.feature.mfcc(x_normalize, sr=sr,n_mfcc=n_mfcc,hop_length=int(0.010*sr), n_fft=int(0.025*sr))
        mfccs = mfccs.T
        print("MFCCS ORIGINAL ", mfccs.shape)
        
        mean_mfccs = np.mean(mfccs, axis=0)
        mean_mfccs = np.reshape(mean_mfccs, (mean_mfccs.shape[0],1))
        
        std_mfcss = np.std(mfccs, axis=0)
        std_mfcss = np.reshape(std_mfcss,(std_mfcss.shape[0],1))
        print("std  ",std_mfcss.shape)
        
        skew_mfccs = skew(mfccs)
        skew_mfccs = np.reshape(skew_mfccs, (skew_mfccs.shape[0],1))
        
        kurt_mfcss = kurtosis(mfccs)
        kurt_mfcss = np.reshape(kurt_mfcss, (kurt_mfcss.shape[0],1))
       
        mfccFull =  np.hstack((mean_mfccs.flatten(),std_mfcss.flatten(),skew_mfccs.flatten(),kurt_mfcss.flatten()))
        mfccFull = np.reshape(mfccFull, (1,mfccFull.shape[0]))
        
        print("Mfcc con mean, std, skew, kurt",mfccFull.shape)
        
        n_windows = mfccs.shape[1]
        path = sound_info[0]
        speaker_id = sound_info[2]
        word=sound_info[1]        
        labelPath = lePath.transform([path])
        labelSpeaker = leSpeaker.transform([speaker_id])
        labelWord= leWord.transform([word])   
        
        #labelPath = np.repeat(labelPath,n_windows)
        #labelWord =  np.repeat(labelWord,n_windows)
        #labelSpeaker= np.repeat(labelSpeaker,n_windows)        
        fila_P_W_S = np.array([labelPath,labelWord,labelSpeaker])  
        fila_P_W_S =fila_P_W_S.T
        print("Filaa ",fila_P_W_S.shape)
        mfccFull =  np.hstack((mfccFull,fila_P_W_S)) 
        print("mfccFull   ",mfccFull.shape)
        matrix_features = np.concatenate((matrix_features,mfccFull), axis=0)    
        print("********** muestra final   ",matrix_features.shape)
        
    return matrix_features[1:,:]

In [124]:
matrixFinal = build_features(path_word_speakerId, 20,NSAMPLES, PATHSAMPLES, SOUNDS)
print(matrixFinal.shape)

MFCCS ORIGINAL  (101, 20)
std   (20, 1)
Mfcc con mean, std, skew, kurt (1, 80)
Filaa  (1, 3)
mfccFull    (1, 83)
********** muestra final    (2, 83)
MFCCS ORIGINAL  (101, 20)
std   (20, 1)
Mfcc con mean, std, skew, kurt (1, 80)
Filaa  (1, 3)
mfccFull    (1, 83)
********** muestra final    (3, 83)
MFCCS ORIGINAL  (101, 20)
std   (20, 1)
Mfcc con mean, std, skew, kurt (1, 80)
Filaa  (1, 3)
mfccFull    (1, 83)
********** muestra final    (4, 83)
(3, 83)
